In [11]:
# !wget --header="Host: drive.usercontent.google.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-US,en;q=0.9,ar;q=0.8" --header="Cookie: HSID=Ag2OIHvsd2Wub4C7z; SSID=AWnBcQKwDHiTrZAU1; APISID=pltrFZgE9lJ0o1gq/AN9feEHYvs8oHd519; SAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; __Secure-1PAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; __Secure-3PAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; SID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-uttBbVDRolhF-hY16nwHXw0gACgYKAWISAQASFQHGX2MivNTw_E_toJuIRy6LMpKNOBoVAUF8yKpFSmvq7AMjvEWeNc50Zff40076; __Secure-1PSID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-utbSY2jBY1VXuw8gYl5hIO2QACgYKAXsSAQASFQHGX2MihVCJ1PwLozGqZgdSatM9QhoVAUF8yKpgrsTvI8i_UE-YHpoN7Gx-0076; __Secure-3PSID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-utwVfPl2imdPimZJ9tdDZGQAACgYKAUESAQASFQHGX2MiEJ49mV4jME2kttDAV5hwWBoVAUF8yKp80mIgju1lu-q4nI7VsFDM0076; NID=511=efI9IZpxtyJ7Dw1MAUXU8FlzS5jXGewY4Er8HliWc3A0RSWdgvNDyKY66ETjgRyTGWPbWODSmiSeYSBab5SPHVwqbJxd6ZeGW2f6BkHi61UKksXPH0CVJRM1hKpMjHPU5qw7tboM2Mi87NrosV8COB-GCLulLLbjOoSAEQewTe8NVZ5Owq8IkwvxFGfJkmUKEMkFWrw9yb5nTDl3wbZEsGFI92iEdNTSxSRovNCIPN2US-SCFdQ0m2BtvwdiWZbgnn7dSQ8yPA145Kk2BA-ATpJNJ6SJHEHLQY-9CPail9D5qgJgxR925EUg5RGCpEu9wS5xbA62KTa19wAvbAq7Dk3TWc-iX4p1s7ESFyDC7yMpFxiFPJjqkWwFi_ZfiK2TW2t0TQ60DFBxqOytQaLyHrkEvD-CQPVj6OCOP22cZY0Cu61HaAQgFO9pXH-kJUlywzVdbirJumN5gswyaQ49b3KdLcG0Jb7brOMTM24T2nGtQ10hJzsnTwX7dBk3ujqQrI_DGuURvPassPUrIZ0; AEC=Ae3NU9MOEGeKAZjP6INpOYbyMraWAWztmx5pJB_1ILu1furiTy1K37k15u0; __Secure-1PSIDTS=sidts-CjEBYfD7Z9twEKTWJ9gU7KG-rLbxJGNRQIoG3wH6JVu6yiCC2fsRrm7tN8L6d5WlILrnEAA; __Secure-3PSIDTS=sidts-CjEBYfD7Z9twEKTWJ9gU7KG-rLbxJGNRQIoG3wH6JVu6yiCC2fsRrm7tN8L6d5WlILrnEAA; 1P_JAR=2024-02-18-08; SIDCC=ABTWhQExCxkfmwCkG1RaEgz8U1ZkPeh3HmLMUdMt8S5cNSsLY5U5rAL6wlvq7dtjRw7zrtAbqsFI; __Secure-1PSIDCC=ABTWhQH0jLeRIS6Tu3LS8DXB5Q3gGDq9LTmlk60FKu795Bf0UbzsOcYWVAE96clq5aAL8i724Q0; __Secure-3PSIDCC=ABTWhQHIFcyv3nZYwp78WXEQal71jCE_ZsGT5lXs8VLr7XDIfFqHcLTIPz4HxzJb9ZnYQ5l2s9eU" --header="Connection: keep-alive" "https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040" -c -O 'tennis_court_det_dataset.zip'
# !unzip tennis_court_det_dataset.zip

# Training Model with PyTorch

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Create Torch Dataset

In [14]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        # Standardise and Normalise Dimensions
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229, 0.224, 0.225]) # Normalise pixels
        ])

    def __len__(self): # Define length of dataset
        return len(self.data) # length of dictionary read
    
    def __getitem__(self, idx):
        # takes an index and returns and image and keypoints
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2] # get height and width

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert from bgr to rgb
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        # Map keypoints to same positions after resize
        kps[::2] *= 224.0 / w # adjust x coordinates to width
        kps[1::2] *= 224.0 / h # adjust y coordinates to height

        return img, kps
        
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset = KeypointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create and Train Model

In [15]:
model = models.resnet50(pretrained=True) # Downloads pretrained CNN
# Replace last layer for fine tuning of Keypoints
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # Replaces last layer for our 14 Keypoints on the tennis court

model = model.to(device)

criterion = torch.nn.MSELoss() #Mean Squared error
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

epochs = 5 # 20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs) # predict the keypoints with model
        loss = criterion(outputs, kps) # calculate the loss
        loss.backward() # backward propagation with MSE
        optimizer.step() # a learning step

        print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14702.0859375
Epoch 0, iter 1, loss: 15246.1572265625
Epoch 0, iter 2, loss: 15086.0009765625
Epoch 0, iter 3, loss: 14606.6650390625
Epoch 0, iter 4, loss: 14010.763671875
Epoch 0, iter 5, loss: 14738.9501953125
Epoch 0, iter 6, loss: 14557.494140625
Epoch 0, iter 7, loss: 14565.033203125
Epoch 0, iter 8, loss: 14917.2294921875
Epoch 0, iter 9, loss: 14125.611328125
Epoch 0, iter 10, loss: 15242.0986328125
Epoch 0, iter 11, loss: 14928.7353515625
Epoch 0, iter 12, loss: 14383.7158203125
Epoch 0, iter 13, loss: 14256.787109375
Epoch 0, iter 14, loss: 14407.115234375
Epoch 0, iter 15, loss: 14834.017578125
Epoch 0, iter 16, loss: 14331.1826171875
Epoch 0, iter 17, loss: 14074.3701171875
Epoch 0, iter 18, loss: 14329.5556640625
Epoch 0, iter 19, loss: 13958.341796875
Epoch 0, iter 20, loss: 13800.9873046875
Epoch 0, iter 21, loss: 13975.9384765625
Epoch 0, iter 22, loss: 14448.8154296875
Epoch 0, iter 23, loss: 13728.8154296875
Epoch 0, iter 24, loss: 13931.3496093

In [17]:
torch.save(model.state_dict(), "keypoints_model.pth")